In [ ]:
import numpy as np
import random

# Bienvenue dans notre programme !

# Trouvez le meilleur trajet en quelques clics

# C'est simple, il suffit de nous donner quelques informations :

### nombre de vèhicule:

In [ ]:
nb_vehicule = int(input("Donner le nombre de vèhicules : "))

Donner le nombre de vèhicules : 3


### capacitè de chaque vèhicule :

In [ ]:
cp_vehicule = float(input("Donner la capacitè des vehicules : "))

Donner la capacitè des vehicules : 21


### nombre de clients :

In [ ]:
nb_client = int(input("Donner le nombre de clients : "))

Donner le nombre de clients : 5


### demande de chaque client :

In [ ]:
demande_client = []
for i in range(1, nb_client+1) :
  demande = float(input(f"Donner la demande de {i}eme client : "))
  demande_client.append(demande)

Donner la demande de 1eme client : 5
Donner la demande de 2eme client : 7
Donner la demande de 3eme client : 10
Donner la demande de 4eme client : 6
Donner la demande de 5eme client : 9


### creation de la matrice des distances :

In [ ]:
np.random.seed(41)
matrice_distance = np.array([[0 if i==j else random.randint(1, 1000) for i in range(nb_client+1)]for j in range(nb_client+1)])
matrice_distance += matrice_distance.T
# on va remplire la diagonale par des 0
np.fill_diagonal(matrice_distance, 0)
print(f"matrice de distance :\n{matrice_distance}")

matrice de distance :
[[   0 1579  894 1286 1245 1213]
 [1579    0  694  969 1190 1188]
 [ 894  694    0 1569 1014 1328]
 [1286  969 1569    0 1070 1028]
 [1245 1190 1014 1070    0 1462]
 [1213 1188 1328 1028 1462    0]]


### nombre d'individus (solutions realisables aleatoires) :

In [ ]:
nb_individus = int(input("Donner le nombre d'individus de la premiere generation : "))

Donner le nombre d'individus de la premiere generation : 8


### creation de la population initiale :

In [ ]:
def PI() :
  population = []
  clients = list(range(1, nb_client+1))
  #On crée une copie de la liste mélangée en utilisant clients[:]
  #et on l'ajoute à la liste matrice.
  #Ceci garantit que chaque sous-liste dans la matrice est une copie séparée de la liste clients mélangée.
  for _ in range(nb_individus) :
    random.shuffle(clients)
    population.append(clients[:])
  return population

### representation des solutions liste en liste :

In [ ]:
def transform(solution, demandes= demande_client, nb_vehicule = nb_vehicule) :
    trajet_v = [[] for _ in range(nb_vehicule)]  # Une liste de listes pour stocker les trajets de chaque véhicule

    cp = cp_vehicule # initialiser la capacitè des vehicules
    vehicule_idx = 0  # Indice du véhicule actuel
    trajet_v[vehicule_idx].append(0) # le premier vehicule commence a partir de depot

    for client in solution :
        if demandes[client - 1] <= cp :  # Vérifiez si la demande du client peut être satisfaite
            cp -= demandes[client - 1]
            trajet_v[vehicule_idx].append(client)

        else :
            # Si la demande du client ne peut pas être satisfaite, passez au véhicule suivant
            trajet_v[vehicule_idx].append(0) # le vehicule retourne au dèpot
            vehicule_idx += 1 # avancer vers le prochaine vehicule
            cp = cp_vehicule  # Réinitialisez la capacité du vehicule
            trajet_v[vehicule_idx].append(0) # le prochain vehicule commence a partir de dèpot
            trajet_v[vehicule_idx].append(client)
            cp -= demandes[client-1]

    for chemin in trajet_v :
        if len(chemin) == 0 :
            chemin.extend([0, 0])
        elif chemin[-1] != 0 :
            chemin.append(0)

    return trajet_v

### calculer la fonction fitness des solutions :

In [ ]:
def fitness(solution, matrice_distance= matrice_distance):
    trajets_vehicules = transform(solution)
    # Calcul du score de fitness
    score = 0
    for trajet in trajets_vehicules:
      for client in range(len(trajet) -1) :
        # Ajouter la distance entre les clients consécutifs dans x
        score += matrice_distance[trajet[client]][trajet[client + 1]] #trajet i cest la ligne de client actuelle et le trajet i+1 cest la colonne de prochain client
    return score

### ecrire les solutions avec leurs scores fitness en liste des dictionnaires :

In [ ]:
def solution_score(solution) :
  dict_solution_score = {}
  score = fitness(solution)
  dict_solution_score["solution"] = solution
  dict_solution_score["score_fitness"] = score
  return dict_solution_score

### faire le croisement pour la creation des nouveaux enfants :

In [ ]:
def croisement(parent1, parent2):
    # Choix du point de croisement
    point_croisement = int(nb_client / 2)

    # Création des solutions filles initialisées avec des valeurs nulles
    enfant1 = [0] * len(parent1)
    enfant2 = [0] * len(parent2)

    # Ajout de la partie avant le point de croisement
    enfant1[:point_croisement] = parent1[:point_croisement]
    enfant2[:point_croisement] = parent2[:point_croisement]

    # Ajout de la partie après le point de croisement, en évitant les répétitions
    enfant1_point_croisement = set(enfant1[:point_croisement])
    enfant2_point_croisement = set(enfant2[:point_croisement])

    for client in parent2:
        if client not in enfant1_point_croisement:
            enfant1[point_croisement] = client
            enfant1_point_croisement.add(client)
            break

    for client in parent1:
        if client not in enfant2_point_croisement:
            enfant2[point_croisement] = client
            enfant2_point_croisement.add(client)
            break

# Compléter les solutions filles avec les éléments manquants
    for i in range(len(parent1)):
        if enfant1[i] == 0:
            for client in parent2:
                if client not in enfant1_point_croisement:
                    enfant1[i] = client
                    enfant1_point_croisement.add(client)
                    break

        if enfant2[i] == 0:
            for client in parent1:
                if client not in enfant2_point_croisement:
                    enfant2[i] = client
                    enfant2_point_croisement.add(client)
                    break

    return enfant1, enfant2


### creation de la nouvelle generation :

In [ ]:
def creation_nouvelle_generation(population) :
  nouvelle_generation = []
  for _ in range(nb_individus) :
      parent1, parent2 = random.sample(population, k= 2) # selection de deux parents aleatoirement
      enfant1, enfant2 = croisement(parent1, parent2) # appelle a la fonction croisement pour generer deux solutions enfants a partir des deux parents
      nouvelle_generation.extend([parent1, parent2, enfant1[:], enfant2[:]]) # ajouter a la nouvelle generation les deux enfants avec leurs parents
  return nouvelle_generation

### trier les solutions de la nouvelle generation en ordre decroissant en fonction de leurs score fitness :

In [ ]:
def selectionGen(generation):
    meilleurs_solutions = sorted(generation, key=lambda x: x['score_fitness'], reverse= True)
    meilleurs_solutions = meilleurs_solutions[:nb_individus]
    return meilleurs_solutions

### faire l'etape de mutation :

In [ ]:
def mutation(generation):
  solution_aleatoire = random.choice(generation) # choisir une solution aleatoire depuis la generation
  client1, client2 = random.sample(range(1, nb_client +1), k= 2) # prendre deux clients(indices de la solution) aleatoirement

  # faire la permutation entre les deux client de la solution
  x = solution_aleatoire["solution"][client1 - 1]
  solution_aleatoire["solution"][client1 - 1] = solution_aleatoire["solution"][client2 - 1]
  solution_aleatoire["solution"][client2 - 1] = x

  # dans cette partie on va calculer le score fitness de la solution apres le changement
  score = fitness(solution_aleatoire['solution'])
  solution_aleatoire["score_fitness"] = score

### le programme principale :

In [30]:
nb_iteration =50# le nombre d'iteration (critère d'arret)
population = PI() # creation de la population initiale


for _ in range(nb_iteration) :
    if _ == 0 :
        generation = creation_nouvelle_generation(population)
    else :
        generation = creation_nouvelle_generation(generation)

    for i, solution in enumerate(generation):
        generation[i] = solution_score(solution)

    generation = selectionGen(generation)
    mutation(generation)
    generation = [solution['solution'] for solution in generation]


solution_optimale = generation[0]
print(f"la solution optimale est {solution_optimale} et son score : {fitness(solution_optimale)}")

la solution optimale est [5, 4, 2, 3, 1] et son score : 10827
